In [9]:
import numpy as np
import pandas as pd

# install pandas_ta - https://github.com/twopirllc/pandas-ta
import pandas_ta as ta

import matplotlib.pyplot as plt

# Calculate Technical Indicators

In [10]:
btc_price_data_1_year = pd.read_csv("data/bitcoin_historical_data_1_year.csv")
btc_price_data_1_year

,timestamp,open,high,low,close,volume,date,time
0,2023-11-01 00:00:00,34618.86,34676.51,34656.38,34667.88,48.953000,2023-11-01,00:00:00
1,2023-11-01 00:01:00,34642.54,34687.53,34673.30,34642.82,16.178075,2023-11-01,00:01:00
2,2023-11-01 00:02:00,34637.97,34656.82,34642.53,34656.56,8.753120,2023-11-01,00:02:00
3,2023-11-01 00:03:00,34617.22,34656.56,34656.56,34629.34,11.308610,2023-11-01,00:03:00
4,2023-11-01 00:04:00,34597.99,34630.42,34629.41,34622.27,8.583808,2023-11-01,00:04:00
...,...,...,...,...,...,...,...,...
528627,2024-10-31 23:56:00,70238.76,70248.97,70248.97,70238.76,1.189134,2024-10-31,23:56:00
528628,2024-10-31 23:57:00,70218.00,70250.00,70238.77,70233.24,4.767082,2024-10-31,23:57:00
528629,2024-10-31 23:58:00,70193.97,70242.25,70232.55,70207.78,9.589688,2024-10-31,23:58:00
528630,2024-10-31 23:59:00,70175.16,70207.79,70207.79,70197.83,7.112237,2024-10-31,23:59:00


Calculating technical indicators like RSI (Relative Strength Index), MACD (Moving Average Convergence Divergence), and Moving Averages can be done with the help of Python's `pandas` library. Additionally, for convenience, the `ta-lib` library or the `pandas_ta` package provides ready-made functions for technical indicators.

Here are methods for calculating each of these indicators manually with `pandas`.

### 1. Relative Strength Index (RSI)

RSI is a momentum oscillator that measures the speed and change of price movements, typically over a 14-period interval.

```python
import pandas as pd

def calculate_rsi(data, window=14):
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    data['RSI'] = rsi
    return data

# Assuming 'data' is a DataFrame with a 'Close' price column
data = pd.DataFrame({'Close': [...]})  # Insert your close price data here
data = calculate_rsi(data)
print(data[['Close', 'RSI']])
```

### 2. Moving Average Convergence Divergence (MACD)

MACD is calculated using two exponential moving averages (EMA): the 12-day EMA and the 26-day EMA, with a 9-day EMA as the signal line.

```python
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    data['EMA12'] = data['Close'].ewm(span=short_window, adjust=False).mean()
    data['EMA26'] = data['Close'].ewm(span=long_window, adjust=False).mean()
    
    # MACD Line
    data['MACD'] = data['EMA12'] - data['EMA26']
    
    # Signal Line
    data['Signal_Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    
    return data

# Assuming 'data' is a DataFrame with a 'Close' price column
data = calculate_macd(data)
print(data[['Close', 'MACD', 'Signal_Line']])
```

### 3. Moving Averages (Simple Moving Average and Exponential Moving Average)

**Simple Moving Average (SMA)** is the average price over a specified number of periods, while **Exponential Moving Average (EMA)** gives more weight to recent prices.

```python
def calculate_moving_averages(data, sma_window=20, ema_window=20):
    # Simple Moving Average
    data['SMA'] = data['Close'].rolling(window=sma_window).mean()
    
    # Exponential Moving Average
    data['EMA'] = data['Close'].ewm(span=ema_window, adjust=False).mean()
    
    return data

# Assuming 'data' is a DataFrame with a 'Close' price column
data = calculate_moving_averages(data)
print(data[['Close', 'SMA', 'EMA']])
```

### Using `pandas_ta` or `ta-lib` (Optional)

If you prefer using pre-built functions, you can install and use libraries like `pandas_ta` or `TA-Lib`. Here's an example with `pandas_ta`:

```python
# Install pandas_ta if not already installed
# !pip install pandas_ta

import pandas as pd
import pandas_ta as ta

# Assuming 'data' is a DataFrame with a 'Close' price column
data['RSI'] = ta.rsi(data['Close'], length=14)
data['MACD'], data['Signal_Line'], _ = ta.macd(data['Close'], fast=12, slow=26, signal=9)
data['SMA'] = ta.sma(data['Close'], length=20)
data['EMA'] = ta.ema(data['Close'], length=20)

print(data[['Close', 'RSI', 'MACD', 'Signal_Line', 'SMA', 'EMA']])
```

Using these methods, you can compute RSI, MACD, and moving averages on stock price data effectively in Python. Let me know if you need further customization or explanations for each indicator!

## Using pandas_ta or ta-lib (Optional)

In [11]:
btc_price_data_1_year['RSI'] = ta.rsi(btc_price_data_1_year['close'], length=14)
btc_price_data_1_year['MACD'], btc_price_data_1_year['Signal_Line'], _ = ta.macd(btc_price_data_1_year['close'], fast=12, slow=26, signal=9)
btc_price_data_1_year['SMA'] = ta.sma(btc_price_data_1_year['close'], length=20)
btc_price_data_1_year['EMA'] = ta.ema(btc_price_data_1_year['close'], length=20)

btc_price_data_1_year[['close', 'RSI', 'MACD', 'Signal_Line', 'SMA', 'EMA']]

,close,RSI,MACD,Signal_Line,SMA,EMA
0,34667.88,NaN,MACD_12_26_9,MACDh_12_26_9,NaN,NaN
1,34642.82,NaN,MACD_12_26_9,MACDh_12_26_9,NaN,NaN
2,34656.56,NaN,MACD_12_26_9,MACDh_12_26_9,NaN,NaN
3,34629.34,NaN,MACD_12_26_9,MACDh_12_26_9,NaN,NaN
4,34622.27,NaN,MACD_12_26_9,MACDh_12_26_9,NaN,NaN
...,...,...,...,...,...,...
528627,70238.76,26.231313,MACD_12_26_9,MACDh_12_26_9,70352.5490,70338.173592
528628,70233.24,25.661202,MACD_12_26_9,MACDh_12_26_9,70338.1685,70328.179916
528629,70207.78,23.160865,MACD_12_26_9,MACDh_12_26_9,70325.1425,70316.713258
528630,70197.83,22.248494,MACD_12_26_9,MACDh_12_26_9,70313.8910,70305.391043
